In [ ]:
import os

def scale_input_data(scale_factor):
  file_bases = ['./input/train_transaction', './input/test_transaction']
  for file_base in file_bases:
    import pandas as pd
    import shutil
    if scale_factor == 1.0:
      shutil.copyfile(file_base + '.csv', file_base + '.scaled.csv')
      continue
    df_to_scale = pd.read_csv(file_base + '.csv')
    new_num_rows = int(scale_factor * len(df_to_scale))
    if scale_factor <= 1.0:
      df_to_scale = df_to_scale.iloc[:new_num_rows]
    else:
      while len(df_to_scale) < new_num_rows:
        df_to_scale = pd.concat([df_to_scale, df_to_scale[:min(new_num_rows - len(df_to_scale), len(df_to_scale))]])
    df_to_scale.to_csv(file_base + '.scaled.csv', index=False)

if 'INPUT_SCALE_FACTOR' in os.environ:
  scale_input_data(float(os.environ['INPUT_SCALE_FACTOR']))

 # 3- KEŞIFSEL VERI ANALIZI (EDA) 

![veritablosu](https://i.pinimg.com/originals/6a/37/fb/6a37fbcea9be5ecf69925610576fcc84.png)

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
exec(os.environ['IREWR_IMPORTS'])
# FIRST-AUTHOR: remove plotting
# from matplotlib import pyplot as plt
# import seaborn as sns
# import missingno as msno
import datetime

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# FIRST-AUTHOR: remove path printing
# import os
# for dirname, _, filenames in os.walk('./input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### 3A-TRANSACTION VE IDENTITY DATASETLERINI TANIYALIM

#### Train Transaction
 590540 rows ve 394 columns dan olusuyor

**TransactionID :** 590540 islem icin 2987000 3577539 arasinda id verilmistir

**isFraud:**  0 normal  işlemi 1 ise dolandırıcılık işlemini temsil eder.

**TransactionDT:** tam belirli olmayan bir tarih saatinden referansla işlemlerin saniye cinsinden timedelta larını içerir. Daha anlamlı hale getirmek için zaman dilimlerine ayırıldığından 190 gunlük işlemleri içerdiği anlaşılıyor.

**TransactionAMT:** USD bazında işlem tutarları değerini içerir. noktadan sonra 3 decimal olan değerlerin başka doviz kurlarında yapılan işlemlerin USD ye değiştirme yapıldığı anlaşılıyor.

**ProductCD:** her işlem için ürün kodu değerlerini içerir - katogorik değişkenler
                w 71%, c 14%,  (h,s,r) 15% 

**card1, card2, card3, card5 :** kartın bankası, ülke vb. gibi ödeme kartı bilgileri.

**card4**  işlemin yapıldığı kredi kartı şirketidir - katogorik değişkenler
            visa 65% MasterCard 32% (discover, american express) 3%
            
**card6** işlemin yapıldığı kart türüdür - katogorik değişkenler
            debit 74% credit 25% (charge card, debit or credit) 1%


**addr1** alici faturalandırma bölgesi adresi
**addr2** alici faturalandırma ülkesi adresi


**dist1:**  59% Nan değeri olan fatura adresi, posta adresi, posta kodu, IP adresi, telefon alanı vb. arasındaki mesafeleri ifade eder.

**dist2:** 94% Nan değeri var silinebilir

**R_emaildomain** alıcı kişinin mail adresi - bağzı işlemlerde alıcı gerek olmadığından 76% Nan değeri var.

P_ and (R__) emaildomain: purchaser and recipient email domain
C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.
D1-D15: timedelta, such as days between previous transaction, etc.d8-d9 float digerleri int degerler

M1-M9: match, such as names on card and address, etc.
             T F or NaN
Column M4 appears to be different with values like M2 and M0

V1-V339: Vesta engineered rich features, including ranking, counting, and other entity relations.

https://www.kaggle.com/kabure/extensive-eda-and-modeling-xgb-hyperopt

In [2]:
# CALISACAGIM DATASET
train_transaction=pd.read_csv('./input/train_transaction.scaled.csv')
test_transaction=pd.read_csv('./input/test_transaction.scaled.csv')

In [3]:
# test_transaction setinde isFraud sutunu bulunmadigindan concat oncesi yeni ekliyoruz ve 2 degerini dolduruyoruz
test_transaction['isFraud']=2
# train_transaction ve test_transaction setlerini concat ediyoruz
transaction=pd.concat([train_transaction, test_transaction], ignore_index=True)

In [4]:
transaction['D1'].head(40)

0      14.0
1       0.0
2       0.0
3     112.0
4       0.0
5       0.0
6       0.0
7       0.0
8       0.0
9      61.0
10      1.0
11      0.0
12     72.0
13     46.0
14      0.0
15      0.0
16      0.0
17      0.0
18      0.0
19     62.0
20      0.0
21    485.0
22      0.0
23      0.0
24     66.0
25      0.0
26      0.0
27      0.0
28      1.0
29      0.0
30    169.0
31      0.0
32     29.0
33    121.0
34    245.0
35    201.0
36    478.0
37      0.0
38    542.0
39      0.0
Name: D1, dtype: float64

In [5]:
# Column ve Degerlerine genel bakis
for col, values in transaction.iteritems():
    num_uniques = values.nunique()
    print ('{name}: {num_unique}'.format(name=col, num_unique=num_uniques))
    print (values.unique())
    print ('\n')

/tmp/ipykernel_321246/2291311977.py:2: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col, values in transaction.iteritems():


TransactionID: 1097231
[2987000 2987001 2987002 ... 4170237 4170238 4170239]


isFraud: 3
[0 1 2]


TransactionDT: 1068035
[   86400    86401    86469 ... 34214326 34214337 34214345]


TransactionAmt: 29806
[ 68.5    29.     59.    ... 186.161 136.117  59.912]


ProductCD: 5
['W' 'H' 'C' 'S' 'R']


card1: 17091
[13926  2755  4663 ...  4525  3755  1350]


card2: 501
[ nan 404. 490. 567. 514. 555. 360. 100. 111. 352. 375. 418. 303. 314.
 543. 583. 148. 321. 269. 361. 272. 399. 569. 453. 417. 512. 545. 266.
 114. 481. 452. 547. 383. 170. 343. 556. 285. 562. 302. 264. 558. 500.
 396. 103. 206. 143. 243. 476. 199. 174. 423. 446. 492. 523. 440. 528.
 161. 535. 354. 117. 455. 325. 158. 268. 122. 479. 147. 215. 480. 265.
 388. 408. 309. 415. 414. 437. 104. 225. 101. 134. 586. 191. 491. 369.
 322. 494. 532. 313. 474. 324. 475. 298. 429. 432. 553. 566. 599. 296.
 251. 310. 242. 204. 250. 270. 346. 316. 194. 587. 390. 135. 536. 254.
 226. 327. 420. 260. 413. 428. 561. 387. 411. 392. 203. 297. 136

C1: 2187
[1.000e+00 2.000e+00 4.000e+00 ... 2.941e+03 2.946e+03 2.950e+03]


C2: 1784
[1.000e+00 5.000e+00 2.000e+00 ... 3.273e+03 3.274e+03 3.275e+03]


C3: 32
[ 0.  1.  8.  3.  2. 16.  4.  9. 10. 12. 11. 13. 14. 15. 17. 18. 19. 20.
 21. 22. 23.  5. 24. 25.  6.  7. 26. 28. 27. 29. nan 30. 31.]


C4: 1532
[0.000e+00 1.000e+00 2.000e+00 ... 1.589e+03 1.595e+03 1.599e+03]


C5: 353
[  0.   2.   1. 168.   3. 121. 142.   5.   6. 128. 118.  33.   4. 103.
   7.  10.  17.   8. 123. 111. 152. 137. 147. 116.  60.  11. 117.  88.
  72.  26.  56.  87. 134.  99.   9.  37.  38.  14.  15.  31. 130.  28.
 170. 208. 101.  62.  84.  96.  92. 136. 107.  32.  82.  97. 154.  19.
  90. 122. 140.  83. 105. 120.  80.  68. 206. 158. 135.  98. 213. 172.
  58.  12.  39.  69.  91. 110.  16.  57. 167.  79. 188. 138. 186. 141.
 119. 180. 155.  40. 113. 174. 106. 108.  29. 199. 144. 185.  54. 192.
 187. 109. 163.  46.  77.  47.  48. 200. 196. 162.  27. 157.  25.  24.
  94. 102. 156. 146.  20. 143. 115. 278. 197. 160

M1: 2
['T' nan 'F']


M2: 2
['T' nan 'F']


M3: 2
['T' nan 'F']


M4: 3
['M2' 'M0' nan 'M1']


M5: 2
['F' 'T' nan]


M6: 2
['T' 'F' nan]


M7: 2
[nan 'F' 'T']


M8: 2
[nan 'F' 'T']


M9: 2
[nan 'F' 'T']


V1: 2
[ 1. nan  0.]


V2: 12
[ 1. nan  2.  3.  0.  4.  5.  6.  7.  8. 10.  9. 11.]


V3: 12
[ 1. nan  2.  3.  0.  4.  5.  6.  7.  8.  9. 10. 11.]


V4: 10
[ 1. nan  2.  0.  3.  4.  5.  6. 10.  8.  9.]


V5: 11
[ 1. nan  2.  0.  3.  4.  5.  6. 10.  8.  9.  7.]


V6: 14
[ 1. nan  2.  3.  0.  4.  5.  6.  7.  9.  8. 11. 10. 12. 13.]


V7: 14
[ 1. nan  2.  3.  0.  4.  5.  6.  7.  9.  8. 11. 10. 12. 13.]


V8: 12
[ 1. nan  2.  3.  0.  6.  4.  5.  8.  7. 10.  9. 11.]


V9: 12
[ 1. nan  2.  3.  0.  4.  6.  5.  8.  7. 10.  9. 11.]


V10: 6
[ 0. nan  1.  2.  3.  4.  5.]


V11: 8
[ 0. nan  1.  2.  4.  3.  5.  6.  7.]


V12: 5
[ 1.  0. nan  2.  3.  4.]


V13: 7
[ 1.  0. nan  2.  3.  4.  5.  6.]


V14: 2
[ 1. nan  0.]


V15: 13
[ 0. nan  1.  2.  3.  4.  5.  6.  7.  8. 10. 11. 12. 13.]


V16: 23
[ 

V99: 89
[ 0. 10.  1.  2.  5.  4.  9.  6.  3. 11. 26. 22.  7. 12. 24. 23. 17. 27.
 15.  8. 18. 16. 25. 28. 19. 20. 13. 21. 29. 30. 31. 32. 33. 14. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53.
 54. 55. 56. 57. 58. 59. 60. 61. 62. 63. 64. 65. 66. 67. 68. 69. 70. 71.
 72. 73. 74. 75. 76. 77. 78. 79. 80. 81. 82. 83. 84. 85. 86. 87. 88. nan]


V100: 31
[ 0.  4.  1.  2.  3.  5. 10.  6.  8.  7.  9. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. nan 29. 30.]


V101: 870
[  0.   1.   2.   3.   8.   4.   5.   6.   7.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  nan  23.  24.  25.  26.
  27.  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.
  41.  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.
  55.  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.
  69.  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.
  83.  84.  85.  86.  87.  88.  89.  90.  91. 

...

[ nan   0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.
 121. 242.  16.  13.  14.  15.  17.  18.  19.  20.  21.  22. 108.  25.
  36.  24.  50.  75.  40.  28.  44.  30.  32.  64. 128. 192.  72.  23.
  26.  27.  29.  31.  33.  34.  35.  37.  38.  39.  41.  42.  43.  45.
  46.  47.  48.  49.  51.  52.  53.  54.  55.  56.  57.  58.  59.  60.
  61.  62.  63.  65.  66.  67.  68.  69.  70.  71.  73.  74. 100.  80.
  81. 150. 363. 162. 243. 324. 270. 255.  88. 261. 125. 175. 200. 484.
 605.  98. 147. 196. 245. 294. 106. 110. 198.  77. 726. 770. 870. 485.
 510. 214.]


V227: 68
[ nan   0.   2.   3.   1.   4.   6.  13.  23.   8.  61.  32.  78.   9.
  17.  36.  84. 130. 213.  42.  40.   5.   7.  73.  74. 199. 202. 360.
 217.  43.  10. 194.  28.  34.  11.  64. 112.  92.  12.  14.  15.  16.
 238.  39.  50.  60.  31.  20.  56.  27.  79.  75.  37.  41.  59.  21.
  26.  25.  24.  53. 131. 106. 100.  89.  29.  87.  22.  33.  19.]


V228: 240
[ nan   1.   4.   5.   6.   2.   3.   7.   8. 

V244: 34
[nan  1.  2.  7.  4.  8.  9. 10. 11. 12. 13.  3.  5.  6. 14. 15. 16. 17.
 18. 19. 20. 21. 22.  0. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33.]


V245: 69
[ nan   1.   2.   0.   3.   7.   4.  13.   5.   6.  24.   8.   9.  59.
  33.  80.  12. 102.  10.  11. 183.  45.  41.  70.  73. 174. 173. 262.
 192.  28. 157.  32.  35. 187.  63.  86.  38.  39.  40.  61.  14.  15.
  16.  17.  18.  19.  20.  25.  21.  22. 199.  46.  29.  47.  27.  43.
  52.  42.  67.  74.  36.  37.  56.  31.  26. 124.  98.  90.  97.  83.]


V246: 225
[ nan   1.   2.   7.   4.   3.   8.   9.  10.  11.  12.  13.   5.   6.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.   0.  46.  47.  48.  49.  50.  51.  52.  53.  54.
  55.  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.
  69.  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.
  83.  84.  85.  86.  87.  88.  89.  90.  9

[        nan  0.         75.88749695 ... 94.06439972 80.78479767
 38.13460159]


V279: 881
[  0.   1.   3.   2.   4.   5.   8.   9.   6.   7.  10.  11.  12.  13.
  32.  30.  14.  15.  16.  17.  18.  26.  28.  31.  29.  nan  36.  20.
  21.  22.  23.  35.  45.  41.  49.  19.  52.  63.  56.  25.  60.  24.
  65.  27.  53.  50.  55.  59.  33.  38.  37.  64.  66.  39.  46.  34.
  40.  44.  43.  47.  42.  48.  51.  54.  57.  58.  61.  62.  67.  68.
  69.  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.
  83.  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.
  97.  98.  99. 100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110.
 111. 112. 113. 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124.
 125. 126. 127. 128. 129. 130. 131. 132. 133. 134. 135. 136. 137. 138.
 139. 140. 141. 142. 143. 144. 145. 146. 147. 148. 149. 150. 151. 152.
 153. 154. 155. 156. 157. 158. 159. 160. 161. 162. 163. 164. 165. 166.
 167. 168. 169. 170. 171. 172. 173. 174. 175. 176. 177. 1

V297: 86
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. nan 10. 11. 12. 13. 14. 15. 16.
 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34.
 35. 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52.
 53. 54. 55. 56. 57. 58. 59. 60. 61. 62. 63. 64. 65. 66. 67. 68. 69. 70.
 71. 72. 73. 74. 75. 76. 77. 78. 79. 80. 81. 82. 83. 84. 85.]


V298: 126
[  0.   1.   2.   6.   3.  11.   4.   7.   5.   8.   9.  17.  39.  10.
  12.  40.  18.  13.  14.  15.  16.  19.  47.  20.  46.  nan  21.  22.
  23.  24.  25.  26.  44.  27.  28.  29.  30.  31.  32.  33.  34.  35.
  36.  38.  37.  41.  42.  43.  45.  48.  49.  50.  51.  52.  53.  54.
  55.  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.
  69.  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.
  83.  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.
  97.  98.  99. 100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110.
 111. 112. 113. 114. 115. 116. 117. 118. 119. 120. 121. 1

[           nan 0.00000000e+00 5.00000000e+01 2.00000000e+02
 1.00000000e+02 3.00000000e+02 5.00000000e+02 8.00000000e+00
 1.50000000e+02 3.00000000e+01 6.00000000e+01 6.50000000e+01
 1.00000000e+01 4.00000000e+01 7.00000000e+01 2.50000000e+01
 7.50000000e+01 1.25000000e+02 4.50000000e+02 1.20000000e+02
 4.00000000e+02 6.00000000e+02 1.50000000e+01 1.00000000e+03
 1.63000000e+03 2.00000000e+01 1.50000000e+03 8.00000000e+01
 2.50000000e+02 4.50000000e+01 4.95000000e+02 5.00000000e+00
 9.00000000e+01 3.50000000e+02 8.00000000e+02 3.75000000e+02
 7.50000000e+02 9.00000000e+02 1.15000000e+02 3.50000000e+01
 4.25000000e+02 8.50000000e+02 1.27500000e+03 1.70000000e+03
 1.25000000e+03 1.20000000e+03 1.30000000e+02 7.00000000e+02
 9.90000000e+01 4.20000000e+02 8.40000000e+02 2.40000000e+02
 1.75000000e+03 2.70000000e+01 6.75000000e+02 1.90000000e+02
 2.25000000e+02 1.05000000e+02 2.20000000e+02 8.52500000e+01
 1.53000000e+03 1.60000000e+03 1.40000000e+03 1.75000000e+02
 6.50000000e+02 6.250000

In [6]:
# NE KADAR ROW-COLINM OLDUGU

print(f'Train Transaction has {transaction.shape[0]} rows and {transaction.shape[1]} columns.')

Train Transaction has 1097231 rows and 394 columns.


In [7]:
#COLUMN ISIMLER
transaction.columns

Index(['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5',
       ...
       'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 'V337', 'V338',
       'V339'],
      dtype='object', length=394)

In [8]:
# BENIM KISIM OLAN COLUMNLARI BASKA BIR VARIABLE ATIYORUZ ve CALISMAMIZDA COPYASINI KULLANIYORUZ
transaction_mycolumns=transaction.iloc[:,0:55]
my_transaction=transaction_mycolumns.copy()

In [9]:
my_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1097231 entries, 0 to 1097230
Data columns (total 55 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   TransactionID   1097231 non-null  int64  
 1   isFraud         1097231 non-null  int64  
 2   TransactionDT   1097231 non-null  int64  
 3   TransactionAmt  1097231 non-null  float64
 4   ProductCD       1097231 non-null  object 
 5   card1           1097231 non-null  int64  
 6   card2           1079644 non-null  float64
 7   card3           1092664 non-null  float64
 8   card4           1092568 non-null  object 
 9   card5           1088425 non-null  float64
 10  card6           1092653 non-null  object 
 11  addr1           965916 non-null   float64
 12  addr2           965916 non-null   float64
 13  dist1           453743 non-null   float64
 14  dist2           74063 non-null    float64
 15  P_emaildomain   933583 non-null   object 
 16  R_emaildomain   273161 non-null   ob

In [10]:
my_transaction.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1097231 entries, 0 to 1097230
Data columns (total 55 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   TransactionID   1097231 non-null  int64  
 1   isFraud         1097231 non-null  int64  
 2   TransactionDT   1097231 non-null  int64  
 3   TransactionAmt  1097231 non-null  float64
 4   ProductCD       1097231 non-null  object 
 5   card1           1097231 non-null  int64  
 6   card2           1079644 non-null  float64
 7   card3           1092664 non-null  float64
 8   card4           1092568 non-null  object 
 9   card5           1088425 non-null  float64
 10  card6           1092653 non-null  object 
 11  addr1           965916 non-null   float64
 12  addr2           965916 non-null   float64
 13  dist1           453743 non-null   float64
 14  dist2           74063 non-null    float64
 15  P_emaildomain   933583 non-null   object 
 16  R_emaildomain   273161 non-null   ob

In [11]:
my_transaction.describe()

,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,...,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15
count,1.097231e+06,1.097231e+06,1.097231e+06,1.097231e+06,1.097231e+06,1.079644e+06,1.092664e+06,1.088425e+06,965916.000000,965916.000000,...,197970.000000,99050.000000,149264.000000,149264.000000,1.008664e+06,641426.000000,133971.000000,185336.000000,177381.000000,996049.000000
mean,3.575333e+06,9.424132e-01,1.640384e+07,1.348879e+02,9.925744e+03,3.630998e+02,1.533556e+02,1.996868e+02,291.241913,86.765369,...,77.771248,53.888168,153.417191,0.557533,1.415345e+02,183.577017,66.116518,18.117435,58.009370,185.152096
std,3.544294e+05,9.888644e-01,1.081612e+07,2.422407e+02,4.893776e+03,1.582079e+02,1.186064e+01,4.093358e+01,101.889537,2.830234,...,176.391818,133.117045,244.722949,0.317121,2.137367e+02,225.954099,153.782459,74.744808,154.868172,239.163890
min,2.987000e+06,0.000000e+00,8.640000e+04,1.800000e-02,1.000000e+03,1.000000e+02,1.000000e+02,1.000000e+02,100.000000,10.000000,...,-83.000000,0.000000,0.000000,0.000000,0.000000e+00,-53.000000,-83.000000,0.000000,-193.000000,-83.000000
25%,3.261308e+06,0.000000e+00,6.651887e+06,4.200000e+01,6.019000e+03,2.100000e+02,1.500000e+02,1.660000e+02,204.000000,87.000000,...,0.000000,0.000000,1.000000,0.208333,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.535615e+06,0.000000e+00,1.449162e+07,6.795000e+01,9.749000e+03,3.610000e+02,1.500000e+02,2.260000e+02,299.000000,87.000000,...,0.000000,0.000000,37.791664,0.666666,1.300000e+01,69.000000,0.000000,0.000000,0.000000,50.000000
75%,3.895932e+06,2.000000e+00,2.651323e+07,1.250000e+02,1.424750e+04,5.120000e+02,1.500000e+02,2.260000e+02,330.000000,87.000000,...,21.000000,18.000000,201.166672,0.833333,2.240000e+02,333.000000,18.000000,0.000000,0.000000,341.000000
max,4.170239e+06,2.000000e+00,3.421434e+07,3.193739e+04,1.839700e+04,6.000000e+02,2.320000e+02,2.370000e+02,540.000000,102.000000,...,1091.000000,1088.000000,2029.583374,0.958333,1.091000e+03,883.000000,879.000000,1066.000000,1085.000000,1091.000000


In [12]:
# kololasyonu anlam ifade eden kisimlari gozlemleyelim
a=my_transaction.corr().abs()<1
b=my_transaction.corr().abs()>0.5
my_transaction.corr().abs()[a&b]


/tmp/ipykernel_321246/1450012837.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  a=my_transaction.corr().abs()<1
/tmp/ipykernel_321246/1450012837.py:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  b=my_transaction.corr().abs()>0.5
/tmp/ipykernel_321246/1450012837.py:4: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  my_transaction.corr().abs()[a&b]


,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,...,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15
TransactionID,NaN,0.885007,0.998633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
isFraud,0.885007,NaN,0.893596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TransactionDT,0.998633,0.893596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TransactionAmt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
card1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
card2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
card3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.612938,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
card5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
addr1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
addr2,NaN,NaN,NaN,NaN,NaN,NaN,0.612938,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# NE KADAR COLUMNDA MISSING VALUE OLDUGU
print(f'There are {my_transaction.isnull().any().sum()} columns in my_train transaction dataset with missing values.')

There are 49 columns in my_train transaction dataset with missing values.


In [14]:
# Eksik verileri oransal bar ile gorsellestirme
# FIRST-AUTHOR: remove plotting
# msno.bar(train_Mytransaction);

In [15]:
# FIRST-AUTHOR: remove plotting
# msno.matrix(my_transaction);

In [16]:
# degiskenler arasindaki yokluk koralasyonu  -1 demek- birinde null varsa digerinde de null her durumda var
# FIRST-AUTHOR: remove plotting
# msno.heatmap(my_transaction);

# Mapping emails

Tüm e-posta alanlarını ilgili kuruluşlara göre gruplayacağım. 300'den az girdiye sahip tüm değerleri "Other" olarak ayarlayacağım.

Company yahoo / ymail / frontier / rocketmail -> Yahoo

hotmail / outlook / live / msn -> Microsoft

icloud / mac / me -> Appe

prodigy / att / sbcglobal-> AT&T

centurylink / embarqmail -> Centurylink

aim / aol -> AOL

twc / charter -> Spectrum

gmail.com / gmail -> Google

In [17]:
emails = {'gmail': 'google', 'att.net': 'att', 'twc.com': 'spectrum', 
          'scranton.edu': 'other', 'optonline.net': 'other', 'hotmail.co.uk': 'microsoft',
          'comcast.net': 'other', 'yahoo.com.mx': 'yahoo', 'yahoo.fr': 'yahoo',
          'yahoo.es': 'yahoo', 'charter.net': 'spectrum', 'live.com': 'microsoft', 
          'aim.com': 'aol', 'hotmail.de': 'microsoft', 'centurylink.net': 'centurylink',
          'gmail.com': 'google', 'me.com': 'apple', 'earthlink.net': 'other', 'gmx.de': 'other',
          'web.de': 'other', 'cfl.rr.com': 'other', 'hotmail.com': 'microsoft', 
          'protonmail.com': 'other', 'hotmail.fr': 'microsoft', 'windstream.net': 'other', 
          'outlook.es': 'microsoft', 'yahoo.co.jp': 'yahoo', 'yahoo.de': 'yahoo',
          'servicios-ta.com': 'other', 'netzero.net': 'other', 'suddenlink.net': 'other',
          'roadrunner.com': 'other', 'sc.rr.com': 'other', 'live.fr': 'microsoft',
          'verizon.net': 'yahoo', 'msn.com': 'microsoft', 'q.com': 'centurylink', 
          'prodigy.net.mx': 'att', 'frontier.com': 'yahoo', 'anonymous.com': 'other', 
          'rocketmail.com': 'yahoo', 'sbcglobal.net': 'att', 'frontiernet.net': 'yahoo', 
          'ymail.com': 'yahoo', 'outlook.com': 'microsoft', 'mail.com': 'other', 
          'bellsouth.net': 'other', 'embarqmail.com': 'centurylink', 'cableone.net': 'other', 
          'hotmail.es': 'microsoft', 'mac.com': 'apple', 'yahoo.co.uk': 'yahoo', 'netzero.com': 'other', 
          'yahoo.com': 'yahoo', 'live.com.mx': 'microsoft', 'ptd.net': 'other', 'cox.net': 'other',
          'aol.com': 'aol', 'juno.com': 'other', 'icloud.com': 'apple'}

us_emails = ['gmail', 'net', 'edu']

# https://www.kaggle.com/c/ieee-fraud-detection/discussion/100499#latest-579654
for c in ['P_emaildomain', 'R_emaildomain']:
    my_transaction[c + '_bin'] = my_transaction[c].map(emails)
    my_transaction[c + '_suffix'] = my_transaction[c].map(lambda x: str(x).split('.')[-1])
    my_transaction[c + '_suffix'] = my_transaction[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')


In [18]:
for column_name in ['P_emaildomain', 'R_emaildomain']:
    my_transaction.drop(column_name, axis=1, inplace=True)

-Kategorik degiskenler icin deger atama
*mod islemine gore-en sik raslanan katogorik degiskene gore deger atama

In [19]:
object_columns_name=my_transaction.select_dtypes(include='object').columns

In [20]:
object_columns_name

Index(['ProductCD', 'card4', 'card6', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7',
       'M8', 'M9', 'P_emaildomain_bin', 'P_emaildomain_suffix',
       'R_emaildomain_bin', 'R_emaildomain_suffix'],
      dtype='object')

In [21]:
for kategorik_sutun in object_columns_name:
    modun_stringdegeri=my_transaction[kategorik_sutun].mode()[0]
    my_transaction[kategorik_sutun].fillna(modun_stringdegeri,inplace=True)

# Kategorik Değerleri Dönüştürme (Frekans)

Değişken Donusumleri -Variable Transformation

In [22]:
# Kategorik degerlerin eksik degerlerini doldurma yontemlerinden Frekansi kullanacagiz
def frekans(data,columns,n_label="NONE"):
    
    for col in columns:
        #data[col].fillna(n_label,inplace=True)
        fq_encode = data[col].value_counts(dropna=False).to_dict()   
        data[col+"_Fr"] = data[col].map(fq_encode)
        data=data.drop(col,axis=1)
    return data

In [23]:
my_transaction = frekans(my_transaction,object_columns_name)

In [24]:
my_transaction.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,...,M4_Fr,M5_Fr,M6_Fr,M7_Fr,M8_Fr,M9_Fr,P_emaildomain_bin_Fr,P_emaildomain_suffix_Fr,R_emaildomain_bin_Fr,R_emaildomain_suffix_Fr
0,2987000,0,86400,68.5,13926,NaN,150.0,142.0,315.0,87.0,...,122947,900269,349499,1025887,904906,1023191,600444,163648,943151,824070
1,2987001,0,86401,29.0,2755,404.0,150.0,102.0,325.0,87.0,...,876978,196962,349499,1025887,904906,1023191,600444,877701,943151,824070
2,2987002,0,86469,59.0,4663,490.0,150.0,166.0,330.0,87.0,...,876978,900269,747732,1025887,904906,74040,112987,877701,943151,824070
3,2987003,0,86499,50.0,18132,567.0,150.0,117.0,476.0,87.0,...,876978,196962,747732,1025887,904906,1023191,197750,877701,943151,824070
4,2987004,0,86506,50.0,4497,514.0,150.0,102.0,420.0,87.0,...,876978,900269,747732,1025887,904906,1023191,600444,877701,943151,824070


In [25]:
my_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1097231 entries, 0 to 1097230
Data columns (total 57 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   TransactionID            1097231 non-null  int64  
 1   isFraud                  1097231 non-null  int64  
 2   TransactionDT            1097231 non-null  int64  
 3   TransactionAmt           1097231 non-null  float64
 4   card1                    1097231 non-null  int64  
 5   card2                    1079644 non-null  float64
 6   card3                    1092664 non-null  float64
 7   card5                    1088425 non-null  float64
 8   addr1                    965916 non-null   float64
 9   addr2                    965916 non-null   float64
 10  dist1                    453743 non-null   float64
 11  dist2                    74063 non-null    float64
 12  C1                       1097228 non-null  float64
 13  C2                       1097228 non-null 

[](http://)# Yeni Öznitelikler Oluşturma (Feature Extraction)

In [26]:
# Preprocess date column
START_DATE = '2017-12-01'
my_transaction = my_transaction.rename(columns={'TransactionDT': 'TransactionDate'})
startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')
my_transaction['TransactionDate'] = my_transaction['TransactionDate'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))


print(my_transaction['TransactionDate'].head())
print(my_transaction['TransactionDate'].tail())

0   2017-12-02 00:00:00
1   2017-12-02 00:00:01
2   2017-12-02 00:01:09
3   2017-12-02 00:01:39
4   2017-12-02 00:01:46
Name: TransactionDate, dtype: datetime64[ns]
1097226   2018-12-31 23:57:59
1097227   2018-12-31 23:58:07
1097228   2018-12-31 23:58:46
1097229   2018-12-31 23:58:57
1097230   2018-12-31 23:59:05
Name: TransactionDate, dtype: datetime64[ns]


In [27]:
my_transaction.columns

Index(['TransactionID', 'isFraud', 'TransactionDate', 'TransactionAmt',
       'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'dist2',
       'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11',
       'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8',
       'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'ProductCD_Fr',
       'card4_Fr', 'card6_Fr', 'M1_Fr', 'M2_Fr', 'M3_Fr', 'M4_Fr', 'M5_Fr',
       'M6_Fr', 'M7_Fr', 'M8_Fr', 'M9_Fr', 'P_emaildomain_bin_Fr',
       'P_emaildomain_suffix_Fr', 'R_emaildomain_bin_Fr',
       'R_emaildomain_suffix_Fr'],
      dtype='object')

# Aykırı/Uç Verileri (Outliers) Tespit Etme

# A-Veri Temizleme (Data Cleaning / Cleasing)
## 3- Outlier Detection- Aykiri gozlem Analizi

In [28]:
# FIRST-AUTHOR: remove plotting
# import seaborn as sns
my_transaction_TransactionAmt_Fraud=my_transaction[['isFraud','TransactionAmt']]
# FIRST-AUTHOR: remove plotting
# fig, ax = plt.subplots(figsize=(5,5))
# sns.heatmap(my_transaction_TransactionAmt_Fraud.corr(), ax=ax,linewidths=.5,annot=True)
# plt.show()
_ = my_transaction_TransactionAmt_Fraud.corr()

In [29]:
# boxplot
# FIRST-AUTHOR: remove plotting
# sns.boxplot(x=my_transaction['TransactionAmt']);
_ = my_transaction['TransactionAmt']

In [30]:
# boxplot da esik deger atama
Q1=my_transaction['TransactionAmt'].quantile(0.25)
Q3=my_transaction['TransactionAmt'].quantile(0.75)
IQR=Q3-Q1
my_TransactionAmt_Alt_Sinir = Q1-2.5*IQR
my_TransactionAmt_Ust_Sinir = Q3 + 2.5*IQR

In [31]:
Q1, Q3 , IQR,my_TransactionAmt_Alt_Sinir,my_TransactionAmt_Ust_Sinir

(42.0, 125.0, 83.0, -165.5, 332.5)

In [32]:
# boxplot ile belirlenen aykiri degerlere erismek
aykiri_TF=(my_transaction['TransactionAmt']<my_TransactionAmt_Alt_Sinir)|(my_transaction['TransactionAmt']>my_TransactionAmt_Ust_Sinir)
my_transaction[aykiri_TF].index

Int64Index([      7,      31,      41,      56,      58,      60,      67,
                 73,      83,      94,
            ...
            1096973, 1096978, 1096980, 1097003, 1097036, 1097056, 1097057,
            1097089, 1097097, 1097125],
           dtype='int64', length=82955)

In [33]:
# baskilama ile ust deger sonrasi degerlere ust degeri, alt deger sonrasina ise alt degeri atama
def AykiriDegeriBaskila(deger):
    
    if deger > my_TransactionAmt_Ust_Sinir:
       
        deger=my_TransactionAmt_Ust_Sinir
    elif deger < my_TransactionAmt_Alt_Sinir:
       
        deger=-my_TransactionAmt_Alt_Sinir
    
    return deger

my_transaction['TransactionAmt'] = my_transaction['TransactionAmt'].apply(lambda x: AykiriDegeriBaskila(x))

# HAKANIN YONTEMDENEME

# EKSIKLERI DOLDURMA


In [34]:
my_transaction.card1.describe()

count    1.097231e+06
mean     9.925744e+03
std      4.893776e+03
min      1.000000e+03
25%      6.019000e+03
50%      9.749000e+03
75%      1.424750e+04
max      1.839700e+04
Name: card1, dtype: float64

# Seting train and test back

In [35]:
# FIRST-AUTHOR: make notebook run
df = my_transaction

df_train, df_test = df[df['isFraud'] != 'test'], df[df['isFraud'] == 2].drop('isFraud', axis=1)